<a href="https://colab.research.google.com/github/miguellucero123/Analisis_est/blob/main/Trabajo_S2_TAC_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importación de bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

# Configuración de la visualización
plt.style.use('seaborn-v0_8-whitegrid') # Cambiado de 'seaborn'
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Definir la ruta al archivo
ruta = '/content/drive/MyDrive/bank-additional-full.csv'

In [ ]:
# Parte 2: Cargar el archivo csv
import pandas as pd

try:
    df = pd.read_csv(ruta, sep=';')
    display(df.head())
    print(f"Dimensión del DataFrame: {df.shape}")
    print(f"Nombre de las columnas: {df.columns.tolist()}")
except FileNotFoundError:
    print("Error: No se encontró 'bank-additional-full.csv'. Por favor, revisa la ruta del archivo.")
    df = pd.DataFrame() # crear un DataFrame vacío para evitar errores después
except pd.errors.ParserError:
    print("Error: No se pudo analizar el archivo CSV. Por favor, verifica el formato y delimitador.")
    df = pd.DataFrame()
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")
    df = pd.DataFrame()

In [ ]:
# Parte 3: Información básica y exploración inicial
df.info()
display(df.describe())

# Explorar columnas categóricas clave
for col in ['y', 'age', 'marital', 'education']:
    print(f"\nConteo de valores para '{col}':")
    display(df[col].value_counts())

# Revisar valores ausentes
print("\nValores nulos por columna:")
display(df.isnull().sum())

# Revisar tipos de datos de columnas relevantes
print("\nTipos de datos de columnas relevantes:")
display(df[['y', 'age', 'marital', 'education']].dtypes)

In [ ]:
# Parte 4: Preprocesamiento
import numpy as np

# 1. Convertir variable objetivo 'y'
# Check if 'y' column exists before processing
if 'y' in df.columns:
    df['conversion'] = df['y'].map({'yes': 1, 'no': 0}).astype(int)
    df = df.drop('y', axis=1)
else:
    # If 'y' column is not found, check if 'conversion' exists (meaning it was likely already processed)
    if 'conversion' not in df.columns:
         print("Error: La columna 'y' o 'conversion' no se encuentra en el DataFrame. Asegúrate de que el archivo CSV se cargó correctamente.")
         # Optionally, you could create an empty 'conversion' column here if needed for later steps
         # df['conversion'] = np.nan # Or a default value
    # If 'conversion' exists, no action is needed for this step as it's already done.


# 2. Crear categorías de grupos de edad
# Define the bins for age groups. Adding a lower bound of 18 as it's the minimum age in the dataset.
bins = [18, 20, 30, 40, 50, 60, 70, np.inf]
labels = ['<20','[20, 30)','[30, 40)','[40, 50)','[50, 60)','[60, 70)','70+'] # Adjusted labels to match bins
df['grupo_edad'] = pd.cut(df['age'], bins=bins, labels=labels, right=False, include_lowest=True) # Added include_lowest=True

# 3. Imputar valores 'unknown' en 'marital' y 'education' con la categoría más frecuente
for col in ['marital', 'education']:
    if col in df.columns and 'unknown' in df[col].unique(): # Check if column exists and contains 'unknown'
        valor_mas_frecuente = df[col].mode()[0]
        df[col] = df[col].replace('unknown', valor_mas_frecuente)

# 4. Asegurar los tipos de datos correctos
# Check if columns exist before changing their type
if 'age' in df.columns:
    df['age'] = df['age'].astype(int)
if 'conversion' in df.columns:
    df['conversion'] = df['conversion'].astype(int)
if 'grupo_edad' in df.columns:
    df['grupo_edad'] = df['grupo_edad'].astype('category')
if 'marital' in df.columns:
    df['marital'] = df['marital'].astype('category')
if 'education' in df.columns:
    df['education'] = df['education'].astype('category')


# Mostrar el DataFrame actualizado
display(df.head())
display(df.info())

In [ ]:
# Parte 5: Métricas de conversión
# 1. Conteo absoluto de conversiones
conteo_conversion_absoluto = df['conversion'].sum()
print(f"Conteo absoluto de conversiones: {conteo_conversion_absoluto}")

# 2. Tasa de conversión general
tasa_conversion_general = (conteo_conversion_absoluto / len(df)) * 100
print(f"Tasa de conversión general: {tasa_conversion_general:.2f}%")

# 3. Tasa de conversión por edad
tasa_conversion_por_edad = df.groupby('age')['conversion'].mean() * 100
print("\nTasa de conversión por edad:")
display(tasa_conversion_por_edad)

# 4. Tasa de conversión por grupo de edad
# Add observed=False to handle categorical grouping explicitly
tasa_conversion_por_grupo_edad = df.groupby('grupo_edad', observed=False)['conversion'].mean() * 100
print("\nTasa de conversión por grupo de edad:")
display(tasa_conversion_por_grupo_edad)

# 5. Tasa de conversión por estado civil
# Add observed=False to handle categorical grouping explicitly
tasa_conversion_por_marital = df.groupby('marital', observed=False)['conversion'].mean() * 100
print("\nTasa de conversión por estado civil:")
display(tasa_conversion_por_marital)

# 6. Tasa de conversión por estado civil y grupo de edad
# Add observed=False to handle categorical grouping explicitly for both columns
tasa_conversion_por_marital_grupo = df.groupby(['marital', 'grupo_edad'], observed=False)['conversion'].mean().unstack() * 100
print("\nTasa de conversión por estado civil y grupo de edad:")
display(tasa_conversion_por_marital_grupo)